In [0]:
import pandas as pd
import numpy as np
import string

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import FeatureHasher


import os, re, math, json, shutil, pprint, datetime
import PIL.Image, PIL.ImageFont, PIL.ImageDraw  # "pip3 install Pillow" or "pip install Pillow" if needed
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.python.platform import tf_logging

In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!ls "/content/drive/My Drive"

 20190703_191246.jpg		   Facultad		       train_limpio.csv
'aTimeLogger backup'		   Papers		       train_nn.csv
'Colab Notebooks'		   Sinc
 descripciones_train_limpias.csv   titulos_train_limpios.csv


In [0]:
train_limpio = pd.read_csv("/content/drive/My Drive/train_nn.csv")

In [0]:
X, y = train_limpio.drop(["precio", "cuantiles"], axis=1),train_limpio["cuantiles"]

In [0]:
y_dum = pd.get_dummies(y)
scaler = StandardScaler()
X = pd.DataFrame(data=scaler.fit_transform(X), columns=X.columns)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y_dum, test_size=0.20, random_state=96969)

In [0]:
for col in X.columns:
  X[col] = X[col].astype(np.float32)

In [0]:
def train_input_fn(X,y, batch_size=2048):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((X,y))
# Shuffle, repeat, and batch the examples.
    dataset = dataset.cache()
    dataset = dataset.shuffle(1000, reshuffle_each_iteration=True)
    dataset = dataset.repeat()
    dataset = dataset.batch(batch_size, drop_remainder=True)
# Return the dataset.
    return dataset

In [0]:
datas = train_input_fn(X_train, y_train)

In [0]:
"""
	Note: This model can be whatever you want it to be.
	Here, I'm building a simple fully-connected network using 
	our distributed training strategy. 
	This essentailly takes our model and makes it 
	compatible to train on a TPU.
	"""
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1012, input_shape=[1086,], activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(500, activation='relu'))
model.add(tf.keras.layers.Dense(50, activation='softmax'))

# compiling the model using the RMSProp optimizer 
# and Sparse Categorical Crossentropy loss
model.compile(
	optimizer=tf.train.AdamOptimizer(learning_rate=0.01, epsilon=0.001, beta2=0.9, use_locking=True),
	loss=tf.keras.losses.categorical_crossentropy,
	metrics=['accuracy']
)

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 1012)              1100044   
_________________________________________________________________
dropout_5 (Dropout)          (None, 1012)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 500)               506500    
_________________________________________________________________
dense_17 (Dense)             (None, 50)                25050     
Total params: 1,631,594
Trainable params: 1,631,594
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(datas, epochs=100, steps_per_epoch=30)

In [0]:
predictions = model.predict(X_test)

In [0]:
preds = [np.argmax(predictions[i]) for i in range(len(predictions))]

In [0]:
pr = y_test.stack()

In [0]:
accuracy_score(pd.Series(pd.Categorical(pr[pr!=0].index.get_level_values(1))), preds)

0.08125

In [0]:
pd.Series(pd.Categorical(pr[pr!=0].index.get_level_values(1)))

0        19
1        34
2        47
3        46
4        25
         ..
47995     4
47996    37
47997     8
47998    19
47999    34
Length: 48000, dtype: category
Categories (50, int64): [1, 2, 3, 4, ..., 47, 48, 49, 50]

In [0]:
preds[:10], preds[-10:]

([18, 33, 44, 47, 17, 24, 48, 49, 41, 43],
 [39, 36, 43, 17, 21, 16, 39, 11, 23, 31])

In [0]:
desc_train = pd.read_csv("/content/drive/My Drive/descripciones_train_limpias.csv")

In [0]:
desc_train = desc_train.fillna("")

In [0]:
hasher = FeatureHasher(n_features=1000, input_type='string', dtype=np.int8)

In [0]:
vectorized_train = hasher.transform(desc.split() for desc in desc_train["descripcion"])

In [0]:
desc_df = pd.DataFrame(vectorized_train.todense(), columns=[f"fh{n}" for n in range(1000)])

In [0]:
train_desc = pd.concat([train_limpio, desc_df], axis=1)

In [0]:
X, y = train_desc.drop(["precio", "cuantiles"], axis=1),train_desc["cuantiles"]

In [0]:
y_dum = pd.get_dummies(y)
scaler = StandardScaler()
X = pd.DataFrame(data=scaler.fit_transform(X), columns=X.columns)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y_dum, test_size=0.20, random_state=96969)

In [0]:
datas = train_input_fn(X_train, y_train)